In [6]:
import tensorflow as tf
from tensorflow.keras.applications import VGG16
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras import layers, models, optimizers
import os
import shutil

# Step 1: Business Understanding (CRISP-DM Step 1)
# Goal: Classify images as 'with_mask' or 'without_mask' using transfer learning with a pretrained VGG16 model.

# Step 2: Data Understanding (CRISP-DM Step 2)
# Clone the dataset repository from GitHub
repo_dir = "Face-Mask-Detection-"
if os.path.exists(repo_dir):
    shutil.rmtree(repo_dir)  # Remove existing directory to avoid conflicts
!git clone https://github.com/chauhanarpit09/Face-Mask-Detection-.git

# Verify dataset paths
possible_dirs = ["Face-Mask-Detection-/Dataset", "Face-Mask-Detection-/data", "Face-Mask-Detection-/Face Mask Dataset"]
base_dir = None
for dir_option in possible_dirs:
    if os.path.exists(dir_option):
        base_dir = dir_option
        break
if base_dir is None:
    raise FileNotFoundError("Dataset directory not found in known paths. Check the repository structure.")

train_dir = os.path.join(base_dir, 'Train')
val_dir = os.path.join(base_dir, 'Validation')
test_dir = os.path.join(base_dir, 'Test')

if not os.path.exists(train_dir):
    train_dir = os.path.join(base_dir, 'train')
if not os.path.exists(val_dir):
    val_dir = os.path.join(base_dir, 'validation')
if not os.path.exists(test_dir):
    test_dir = os.path.join(base_dir, 'test')

if not os.path.exists(train_dir) or not os.path.exists(val_dir) or not os.path.exists(test_dir):
    raise FileNotFoundError("One or more dataset directories (Train, Validation, Test) are missing.")

# Step 3: Data Preparation (CRISP-DM Step 3)
# Prepare ImageDataGenerators
train_datagen = ImageDataGenerator(rescale=1.0/255.0)
val_datagen = ImageDataGenerator(rescale=1.0/255.0)
test_datagen = ImageDataGenerator(rescale=1.0/255.0)

train_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=(224, 224),
    batch_size=32,
    class_mode='binary')

val_generator = val_datagen.flow_from_directory(
    val_dir,
    target_size=(224, 224),
    batch_size=32,
    class_mode='binary')

test_generator = test_datagen.flow_from_directory(
    test_dir,
    target_size=(224, 224),
    batch_size=32,
    class_mode='binary')

# Step 4: Modeling (CRISP-DM Step 4)
# Load pretrained VGG16 model without the top layer
vgg16_base = VGG16(weights='imagenet', include_top=False, input_shape=(224, 224, 3))

# Freeze the base model
vgg16_base.trainable = False

# Build the model
model = models.Sequential([
    vgg16_base,
    layers.Flatten(),
    layers.Dense(256, activation='relu'),
    layers.Dropout(0.5),
    layers.Dense(1, activation='sigmoid')
])

# Compile the model
model.compile(optimizer=optimizers.Adam(learning_rate=0.0001),
              loss='binary_crossentropy',
              metrics=['accuracy'])

# Step 5: Evaluation (CRISP-DM Step 5)
# Train the model
history = model.fit(
    train_generator,
    epochs=10,
    validation_data=val_generator)

# Evaluate on the test set
test_loss, test_accuracy = model.evaluate(test_generator)
print(f"Test Accuracy: {test_accuracy:.2f}")

# Step 6: Deployment (CRISP-DM Step 6)
def test_image(image_url, model, class_names):
    import requests
    from PIL import Image
    from io import BytesIO

    response = requests.get(image_url)
    img = Image.open(BytesIO(response.content)).resize((224, 224))
    img_array = tf.keras.preprocessing.image.img_to_array(img) / 255.0
    img_array = tf.expand_dims(img_array, 0)  # Add batch dimension

    predictions = model.predict(img_array)
    predicted_class = class_names[int(predictions[0] > 0.5)]
    print(f"Predicted class: {predicted_class}")

# Example Usage
image_url = input("Enter image URL: ")
test_image(image_url, model, ['without_mask', 'with_mask'])


Cloning into 'Face-Mask-Detection-'...
remote: Enumerating objects: 4547, done.
remote: Counting objects: 100% (24/24), done.
remote: Compressing objects: 100% (24/24), done.
remote: Total 4547 (delta 11), reused 0 (delta 0), pack-reused 4523 (from 1)
Receiving objects: 100% (4547/4547), 176.67 MiB | 24.90 MiB/s, done.
Resolving deltas: 100% (14/14), done.
Updating files: 100% (4678/4678), done.


FileNotFoundError: Dataset directory not found in known paths. Check the repository structure.

In [3]:
import os
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D
from tensorflow.keras.models import Model

# Find dataset directory
def find_dataset_dir(base_path):
    for root, dirs, files in os.walk(base_path):
        if 'train' in dirs and 'test' in dirs:
            return root
    return None

# Clone repository if not exists
if not os.path.exists('/content/Face-Mask-Detection-'):
    !git clone https://github.com/chauhanarpit09/Face-Mask-Detection-.git

# Find dataset directory
dataset_base_path = find_dataset_dir('/content/Face-Mask-Detection-')

if dataset_base_path:
    train_dir = os.path.join(dataset_base_path, 'train')
    validation_dir = os.path.join(dataset_base_path, 'test')
else:
    # Fallback to manual path specification
    train_dir = '/content/Face-Mask-Detection-/train'
    validation_dir = '/content/Face-Mask-Detection-/test'

print(f"Train Directory: {train_dir}")
print(f"Validation Directory: {validation_dir}")

# Verify directories exist
print("Train Directory Contents:")
print(os.listdir(train_dir))
print("\nValidation Directory Contents:")
print(os.listdir(validation_dir))

# Data Generators
train_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    validation_split=0.2
)

# Create generators
train_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=(224, 224),
    batch_size=32,
    class_mode='binary',
    subset='training'
)

validation_generator = train_datagen.flow_from_directory(
    validation_dir,
    target_size=(224, 224),
    batch_size=32,
    class_mode='binary',
    subset='validation'
)

Train Directory: /content/Face-Mask-Detection-/facemask/train
Validation Directory: /content/Face-Mask-Detection-/facemask/test
Train Directory Contents:
['without_mask', 'with_mask']

Validation Directory Contents:
['without_mask', 'with_mask']
Found 3437 images belonging to 2 classes.
Found 14 images belonging to 2 classes.


In [1]:
# Face Mask Detection Project Setup and Diagnostics

# 1. Install necessary libraries
!pip install tensorflow keras numpy matplotlib

import os
import sys
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D
from tensorflow.keras.models import Model

# 2. Clone the repository
!git clone https://github.com/chauhanarpit09/Face-Mask-Detection-.git

# 3. Diagnostic Function to Explore Repository
def explore_repository(base_path):
    print("🔍 Repository Structure Exploration:")
    for root, dirs, files in os.walk(base_path):
        level = root.replace(base_path, '').count(os.sep)
        indent = ' ' * 4 * level
        print(f"{indent}{os.path.basename(root)}/")
        subindent = ' ' * 4 * (level + 1)
        for dir in dirs:
            print(f"{subindent}{dir}/")
        for file in files[:5]:  # Limit to first 5 files
            print(f"{subindent}{file}")

# 4. Find Dataset Directory
def find_dataset_directories(base_path):
    dataset_dirs = []
    for root, dirs, files in os.walk(base_path):
        if 'train' in dirs or 'test' in dirs:
            dataset_dirs.append(root)
    return dataset_dirs

# 5. Repository Exploration
print("📂 Exploring Repository Directories:")
explore_repository('/content/Face-Mask-Detection-')

# 6. Find Potential Dataset Directories
potential_dataset_paths = find_dataset_directories('/content/Face-Mask-Detection-')
print("\n📊 Potential Dataset Paths:")
for path in potential_dataset_paths:
    print(path)

# 7. Attempt to Locate Train and Test Directories
def locate_train_test_dirs(base_path):
    train_dirs = []
    test_dirs = []
    for root, dirs, files in os.walk(base_path):
        if 'train' in dirs:
            train_dirs.append(os.path.join(root, 'train'))
        if 'test' in dirs:
            test_dirs.append(os.path.join(root, 'test'))
    return train_dirs, test_dirs

train_directories, test_directories = locate_train_test_dirs('/content/Face-Mask-Detection-')

print("\n🔬 Detected Train Directories:")
for dir in train_directories:
    print(dir)
    print("  Contents:", os.listdir(dir)[:5])

print("\n🔬 Detected Test Directories:")
for dir in test_directories:
    print(dir)
    print("  Contents:", os.listdir(dir)[:5])

# If multiple paths found, we'll use the first valid path
train_dir = train_directories[0] if train_directories else None
test_dir = test_directories[0] if test_directories else None

# 8. Data Generator Setup (Flexible)
if train_dir and test_dir:
    print("\n🚀 Setting Up Data Generators...")
    train_datagen = ImageDataGenerator(
        rescale=1./255,
        rotation_range=20,
        width_shift_range=0.2,
        height_shift_range=0.2,
        shear_range=0.2,
        zoom_range=0.2,
        horizontal_flip=True,
        validation_split=0.2
    )

    train_generator = train_datagen.flow_from_directory(
        train_dir,
        target_size=(224, 224),
        batch_size=32,
        class_mode='binary',
        subset='training'
    )

    validation_generator = train_datagen.flow_from_directory(
        test_dir,
        target_size=(224, 224),
        batch_size=32,
        class_mode='binary',
        subset='validation'
    )

    # 9. Model Building
    base_model = MobileNetV2(
        weights='imagenet',
        include_top=False,
        input_shape=(224, 224, 3)
    )
    base_model.trainable = False

    x = base_model.output
    x = GlobalAveragePooling2D()(x)
    x = Dense(128, activation='relu')(x)
    output = Dense(1, activation='sigmoid')(x)

    model = Model(inputs=base_model.input, outputs=output)

    # 10. Model Compilation
    model.compile(
        optimizer='adam',
        loss='binary_crossentropy',
        metrics=['accuracy']
    )

    # 11. Model Training
    print("\n🏋️ Training Model...")
    history = model.fit(
        train_generator,
        steps_per_epoch=train_generator.samples // train_generator.batch_size,
        validation_data=validation_generator,
        validation_steps=validation_generator.samples // validation_generator.batch_size,
        epochs=5
    )

    # 12. Model Evaluation
    print("\n📊 Model Evaluation:")
    evaluation = model.evaluate(validation_generator)
    print(f"Test Loss: {evaluation[0]}")
    print(f"Test Accuracy: {evaluation[1]}")
else:
    print("❌ Could not locate train and test directories!")

Cloning into 'Face-Mask-Detection-'...
remote: Enumerating objects: 4547, done.
remote: Counting objects: 100% (24/24), done.
remote: Compressing objects: 100% (24/24), done.
remote: Total 4547 (delta 11), reused 0 (delta 0), pack-reused 4523 (from 1)
Receiving objects: 100% (4547/4547), 176.67 MiB | 23.03 MiB/s, done.
Resolving deltas: 100% (14/14), done.
Updating files: 100% (4678/4678), done.
📂 Exploring Repository Directories:
🔍 Repository Structure Exploration:
Face-Mask-Detection-/
    face_detector/
    facemask/
    .git/
    README.md
    facemaskdetector.py
    acc or loss plot.png
    facemask.ipynb
    Confusion matrix.png
    face_detector/
        deploy.prototxt
        res10_300x300_ssd_iter_140000.caffemodel
    facemask/
        train/
        valid/
        test/
        train/
            without_mask/
            with_mask/
            without_mask/
                augmented_image_251.jpg
                1_0_aidai_0058.jpg
                1_0_luojin_0044.jpg
      

/usr/local/lib/python3.10/dist-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:122: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


 19/107 ━━━━━━━━━━━━━━━━━━━━ 32s 369ms/step - accuracy: 0.8065 - loss: 0.3649

/usr/local/lib/python3.10/dist-packages/PIL/Image.py:1054: UserWarning: Palette images with Transparency expressed in bytes should be converted to RGBA images
  warnings.warn(


107/107 ━━━━━━━━━━━━━━━━━━━━ 65s 485ms/step - accuracy: 0.9291 - loss: 0.1545 - val_accuracy: 1.0000 - val_loss: 0.0044
Epoch 2/5
  1/107 ━━━━━━━━━━━━━━━━━━━━ 4s 39ms/step - accuracy: 0.9688 - loss: 0.0467

/usr/lib/python3.10/contextlib.py:153: UserWarning: Your input ran out of data; interrupting training. Make sure that your dataset or generator can generate at least `steps_per_epoch * epochs` batches. You may need to use the `.repeat()` function when building your dataset.
  self.gen.throw(typ, value, traceback)


107/107 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.9688 - loss: 0.0467 - val_accuracy: 1.0000 - val_loss: 0.0022
Epoch 3/5
107/107 ━━━━━━━━━━━━━━━━━━━━ 66s 395ms/step - accuracy: 0.9816 - loss: 0.0437 - val_accuracy: 1.0000 - val_loss: 0.0060
Epoch 4/5
107/107 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 1.0000 - loss: 0.0094 - val_accuracy: 0.9286 - val_loss: 0.1819
Epoch 5/5
107/107 ━━━━━━━━━━━━━━━━━━━━ 82s 405ms/step - accuracy: 0.9869 - loss: 0.0351 - val_accuracy: 0.9286 - val_loss: 0.4690

📊 Model Evaluation:
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 220ms/step - accuracy: 1.0000 - loss: 0.0193
Test Loss: 0.019338520243763924
Test Accuracy: 1.0
